<a href="https://colab.research.google.com/github/analyticsworld1/1-ProPub/blob/main/CatBoost_for_Wind_Power_Prediction_(R%C2%B2_%3D_98_54_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'wind-turbine-power-output-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5675285%2F9364667%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240912%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240912T140955Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D806f94826b92fd61e4ae55e95f3d0cfbbb11dea22320e1d89d9a5cbd185a7c183444c67d3219d0e51f27ccb2e365c87d58682e148b9e7dfc29d88071c0a8fff1260f55191f67a7f806553ea1dbe470419aab7f7c2dc4d15c2cde1f7b8ccd74f91c6bd6c63c96881039a689d37a3e3aa3dbfbeaf31b8cc0b14b587ce8f9926705ccc41aada06d788d388c250f186d9b92fcea5a98ad232bc8502ca7c19c1716ffc5e9e9995676728cae1f569ed8185214dfb3c12f63578bd33cd10cf2ef671024b7d21d107ea9c6e77f574cc2f7a49988e9fceb7ff7510f117a49af4d87ab800c36dabf482ab22c60fd87b7f68e499722b59afb3681523873fb9dc8e854709105'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.precision', 2)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time

# Load the dataset
df = pd.read_csv("/kaggle/input/wind-turbine-power-output-data/T1.csv")
df = df.sort_values(ascending=True,by='Date/Time')
# Replce -ve "LV ActivePower (kW)" values with zero
df["LV ActivePower (kW)"] = df["LV ActivePower (kW)"].apply(lambda x: 0 if x < 0 else x)
display(df.shape)
df.head()

In [ ]:
print(df['Date/Time'].head().tolist())

In [ ]:
duplicate = df[df.isnull().any(axis=1)]
duplicate

In [ ]:
import plotly.graph_objects as go

# Create a scatter plot trace
trace = go.Scatter(
    x=df['Wind Speed (m/s)'],
    y=df['Theoretical_Power_Curve (KWh)'],
    mode='markers',
    marker=dict(size=10, opacity=0.5),
    name='Theoretical Power Curve'
)

# Create the figure object
fig = go.Figure(data=[trace])

# Set axis titles and title
fig.update_layout(
    xaxis_title='Wind Speed (m/s)',
    yaxis_title='Power (kW)',
    title='Power Curves',
    legend_title_text='Legend',
    width=1200,
    height=600
)

# Add gridlines
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

# Display the plot
fig.show()

<span class="label label-default" style="background-color:#DC1010; border-radius:12px; font-weight: bold; font-family:Verdana; font-size:32px; color:#FBFAFC; ">Cut-in Speed and Rated Velocity in Wind Turbines 📊📈</span>

**Cut-in speed is the minimum wind speed required for a wind turbine to start generating electricity.**

**In our case, the cut-in speed is 3.00 m/s. This means that the turbine needs a wind speed of at least 3 meters per second to begin producing power.**

Rated velocity is the wind speed at which the turbine reaches its maximum power output. In 0ur case, the rated velocity is 17.90 m/s. This means that when the wind speed reaches 17.90 meters per second, the turbine will be producing as much power as it is designed to generate.

**Impact on Power Generation:**

- **Cut-in speed:** A lower cut-in speed is generally desirable as it allows the turbine to start generating power in a wider range of wind conditions, improving its overall energy capture efficiency.

- **Rated velocity:** A higher rated velocity means that the turbine can maintain its maximum power output in stronger winds, increasing its energy production potential.

**Relationship between the two:**

The relationship between cut-in speed and rated velocity is important in determining the overall efficiency and performance of a wind turbine. A turbine with a low cut-in speed and a high rated velocity is generally considered more efficient as it can capture more energy from a wider range of wind speeds. However, other factors such as turbine design, blade efficiency, and site-specific conditions also play a significant role in determining the overall power output of a wind turbine.

<center> <img src="https://www.researchgate.net/publication/333440886/figure/fig1/AS:763721878081539@1559096893608/The-output-power-curve-of-a-wind-turbine-according-to-the-wind-speed.png" width="600" height="200"> </center>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Determine cut-in speed
cut_in_speed = df[df['Theoretical_Power_Curve (KWh)'] > 0]['Wind Speed (m/s)'].min()
cut_in_speed = round(cut_in_speed, 1)

# Filter data based on cut-in speed
filtered_data = df[df['Wind Speed (m/s)'] >= cut_in_speed]

# Determine maximum power output
max_power = df['LV ActivePower (kW)'].max()

# Determine rated velocity
rated_velocity = df[df['LV ActivePower (kW)'] >= 0.95 * max_power]['Wind Speed (m/s)'].min()
rated_velocity = round(rated_velocity, 1)

# Determine cut-out speed
cut_out_speed = df[df['Theoretical_Power_Curve (KWh)'] > 0]['Wind Speed (m/s)'].max()
cut_out_speed = round(cut_out_speed, 1)

# Print results
print(f"\033[1;32m\033[1mThe cut-in speed is {cut_in_speed:.2f} m/s\033[0m")
print(f"\033[1;33m\033[1mThe rated velocity is {rated_velocity:.2f} m/s\033[0m")
print(f"\033[1;34m\033[1mThe cut-out speed is {cut_out_speed:.2f} m/s\033[0m")
print(f"\033[1;35m\033[1mThe Maximum Power is {max_power:.2f} m/s\033[0m")

# Create a scatter plot to visualize the power curve
plt.figure(figsize=(20, 8))
plt.scatter(df['Wind Speed (m/s)'], df['Theoretical_Power_Curve (KWh)'], s=10, alpha=0.5, label='Theoretical Power Curve')
plt.xlabel('Wind Speed (m/s)', fontsize=18, color='Blue')
plt.ylabel('Power (kW)', fontsize=18, color='Brown')
plt.ylim([-500, 4000])
plt.title('Power Curve', fontsize=24, color='Brown', pad=20, fontname='Lucida Console')
plt.xticks(fontsize=16, color='orange')
plt.yticks(fontsize=16, color='magenta')
plt.legend()
plt.grid(True)

# Add a vertical line to indicate the cut-in speed
plt.axvline(x=cut_in_speed, color='red', linestyle='--', label='Cut-in Speed', linewidth=3)
plt.text(cut_in_speed, plt.ylim()[1] * 0.7, f'Cut-in Speed: {cut_in_speed} m/s', color='red', ha='right', va='top', fontsize=18, rotation=90)

# Add a vertical line to indicate the rated speed
plt.axvline(x=rated_velocity, color='green', linestyle='--', label='Rated Speed', linewidth=3)
plt.text(rated_velocity, plt.ylim()[1] * 0.7, f'Rated Speed: {rated_velocity} m/s', color='green', ha='right', va='top', fontsize=18, rotation=90)

# Add a vertical line to indicate the cut-out speed
plt.axvline(x=cut_out_speed, color='purple', linestyle='--', label='Cut-out Speed', linewidth=3)
plt.text(cut_out_speed, plt.ylim()[1] * 0.7, f'Cut-out Speed: {cut_out_speed} m/s', color='purple', ha='right', va='top', fontsize=18, rotation=90)

# Add a horizontal line to indicate the maximum power
plt.axhline(y=max_power, color='blue', linestyle='--', label='Maximum Power', linewidth=3)
plt.text(plt.xlim()[0] * 1.1, max_power, f'Maximum Power: {max_power:.2f} kW', color='blue', ha='left', va='bottom', fontsize=18)

plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Convert "Date/Time" column to datetime format
df['Date/Time'] = pd.to_datetime(df['Date/Time'], format="%d %m %Y %H:%M")

# Extract week, month, and hour information
df['Week'] = df['Date/Time'].dt.isocalendar().week
df['Month'] = df['Date/Time'].dt.month
df['Hour'] = df['Date/Time'].dt.hour

# Define function to get season based on Turkey's calendar
def get_season_turkey(month):
    if month in [1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"
    else:
        return "Winter"  # December is considered Winter in Turkey

df['Season'] = df['Month'].apply(get_season_turkey)

# Display the updated DataFrame
df.head()

In [ ]:
plt.figure(figsize=(20, 8))
plt.scatter(df['Wind Speed (m/s)'], df['LV ActivePower (kW)'],color='#FF8C00', s=10, alpha=0.5)
plt.xlabel('Wind Speed (m/s)',fontsize=20, color='Blue')
plt.ylabel('LV ActivePower (kW)',fontsize=20, color='Blue')
plt.title('Power Curve',fontsize=24, color='Blue')
plt.xticks(fontsize=16, color='Red')
plt.yticks(fontsize=16, color='Red')
plt.grid(True)
plt.show()

**Interpreting the Power Curve:**

- **Efficiency:** A steeper slope at lower wind speeds suggests a more efficient turbine in capturing energy from lower wind conditions.
- **Rated Power:** A higher rated power indicates the turbine's maximum energy production capacity.
- **Cut-in Speed:** A lower cut-in speed allows the turbine to start generating power earlier, improving its overall energy capture.
- **Data Variability:** The scatter points around the curve provide insights into the turbine's performance variability under different conditions.

In [ ]:
!pip install astral pytz

**Add Day/Night feature**

We do not know the specific location of the turbine to determine the Sunset/Sunrise time, But... form the wind atlas of Turkey, high Wind Speeds (>10m/s) occurs mostly at the west coast at izmir

In [ ]:
from astral import LocationInfo
from astral.sun import sun
import pytz
from datetime import datetime

# Define location information outside the function
location = LocationInfo("Izmir", "Turkey", "Europe/Istanbul", 38.4192, 27.1287)
timezone = pytz.timezone('Europe/Istanbul')

def is_day_or_night(dt):
    """
    Efficiently determines if it's day or night based on sunrise/sunset.

    Args:
        dt (datetime or Timestamp): The datetime object to check.

    Returns:
        int: 0 for Day, 1 for Night.
    """
    # Ensure dt is a timezone-aware datetime object
    if dt.tzinfo is None:
        dt = timezone.localize(dt.to_pydatetime())
    else:
        dt = dt.astimezone(timezone)

    # Call sun and access sunrise/sunset attributes
    solar_info = sun(location.observer, date=dt.date())
    sunrise = solar_info['sunrise'].astimezone(timezone)
    sunset = solar_info['sunset'].astimezone(timezone)

    return 0 if sunrise <= dt <= sunset else 1  # Daytime or Nighttime

# Apply the function to the 'Date/Time' column
df['Day/Night'] = df['Date/Time'].apply(is_day_or_night)

display(df['Day/Night'].value_counts())
df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Determine cut-in speed
cut_in_speed = df[df['Theoretical_Power_Curve (KWh)'] > 0]['Wind Speed (m/s)'].min()
cut_in_speed = round(cut_in_speed, 1)

# Filter data for day and night
day_data = df[df['Day/Night'] == 0]
night_data = df[df['Day/Night'] == 1]

# Determine maximum power for day and night (optional)
max_power_day = day_data['LV ActivePower (kW)'].max()
max_power_night = night_data['LV ActivePower (kW)'].max()

# Determine rated velocity for day and night (optional)
rated_velocity_day = day_data[day_data['LV ActivePower (kW)'] >= 0.95 * max_power_day]['Wind Speed (m/s)'].min()
rated_velocity_day = round(rated_velocity_day, 1)
rated_velocity_night = night_data[night_data['LV ActivePower (kW)'] >= 0.95 * max_power_night]['Wind Speed (m/s)'].min()
rated_velocity_night = round(rated_velocity_night, 1)

# Determine cut-out speed
cut_out_speed = df[df['Theoretical_Power_Curve (KWh)'] > 0]['Wind Speed (m/s)'].max()
cut_out_speed = round(cut_out_speed, 1)

# Print results
print(f"\033[1;32m\033[1mThe cut-in speed is {cut_in_speed:.2f} m/s\033[0m")
print(f"\033[1;33m\033[1mThe rated velocity (Day): {rated_velocity_day:.2f} m/s\033[0m")
print(f"\033[1;33m\033[1mThe rated velocity (Night): {rated_velocity_night:.2f} m/s\033[0m" if rated_velocity_day is not None and rated_velocity_night is not None else "")
print(f"\033[1;34m\033[1mThe cut-out speed is {cut_out_speed:.2f} m/s\033[0m")
print(f"\033[1;35m\033[1mThe Maximum Power (Day): {max_power_day:.2f} kW (if calculated)\033[0m")
print(f"\033[1;35m\033[1mThe Maximum Power (Night): {max_power_night:.2f} kW (if calculated)\033[0m")

# Create separate scatter plots for day and night data
plt.figure(figsize=(20, 12))

# Day data plot
plt.subplot(211)  # Create subplot for day data at position 1 (top)
plt.scatter(day_data['Wind Speed (m/s)'], day_data['Theoretical_Power_Curve (KWh)'], s=10, alpha=0.5, label='Theoretical Power Curve (Day)')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Power (kW)')
plt.ylim([-500, 4000])
plt.title('Power Curve (Day)', pad=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=10)
plt.grid(True)

# Add lines for day data (optional)
if cut_in_speed is not None:
    plt.axvline(x=cut_in_speed, color='red', linestyle='--', label='Cut-in Speed', linewidth=2)
if rated_velocity_day is not None:
    plt.axvline(x=rated_velocity_day, color='green', linestyle='--', label='Rated Speed', linewidth=2)
if cut_out_speed is not None:
    plt.axvline(x=cut_out_speed, color='purple', linestyle='--', label='Cut-out Speed', linewidth=2)
if max_power_day is not None:
    plt.axhline(y=max_power_day, color='blue', linestyle='--', label='Maximum Power', linewidth=2)

# Night data plot
plt.subplot(212)  # Create subplot for night data at position 2 (bottom)
plt.scatter(night_data['Wind Speed (m/s)'], night_data['Theoretical_Power_Curve (KWh)'], s=10, alpha=0.5, label='Theoretical Power Curve (Night)')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Power (kW)')
plt.ylim([-500, 4000])
plt.title('Power Curve (Night)', pad=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=10)
plt.grid(True)

# Add lines for night data (optional)
if cut_in_speed is not None:
    plt.axvline(x=cut_in_speed, color='red', linestyle='--', label='Cut-in Speed', linewidth=2)
if rated_velocity_night is not None:
    plt.axvline(x=rated_velocity_night, color='green', linestyle='--', label='Rated Speed', linewidth=2)
if cut_out_speed is not None:
    plt.axvline(x=cut_out_speed, color='purple', linestyle='--', label='Cut-out Speed', linewidth=2)
if max_power_night is not None:
    plt.axhline(y=max_power_night, color='blue', linestyle='--', label='Maximum Power', linewidth=2)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

**Add Temperature feature**

Since we have date/time & approximate location, why not making Temperature column?, Since air density is a variable and there's an invesrse relation between air density and temperature

$$
P_{\text{turbine}} = \frac{1}{2} \rho A v^3
$$

$$
\rho \alpha \frac{1}{T}
$$

Here's a breakdown of the variables in the equation:

- **P_turbine:** The power output of the turbine (in watts).
- **ρ:** The air density (in kilograms per cubic meter).
- **A:** The swept area of the turbine blades (in square meters).
- **v:** The wind speed (in meters per second).
- **T:** Temperature (in Kelvin)

The equation is the fundamental equation for calculating the power output of a wind turbine. It represents the Betz limit, which defines the maximum theoretical power that can be extracted from the wind.

In [ ]:
!pip install meteostat

In [ ]:
import pandas as pd
from datetime import datetime
from meteostat import Point, Hourly

# Define the location and time period
location = Point(38.4192, 27.1287)  # Coordinates for Izmir, Turkey

# Fetch historical weather data for the entire period
data_hourly = Hourly(location, df['Date/Time'].min(), df['Date/Time'].max())
data_hourly = data_hourly.fetch()

# Ensure the index is a DatetimeIndex and remove timezone information
data_hourly.index = data_hourly.index.tz_localize(None)

# Resample the hourly data to 10-minute intervals using forward fill
data_10min = data_hourly.resample('10T').ffill().reset_index()

# Select relevant columns and rename them to match the original dataset format
weather_df = data_10min[['time', 'temp', 'rhum', 'pres']]
weather_df = weather_df.rename(columns={
    'time': 'Date/Time',
    'temp': 'Temperature(°C)',
    'rhum': 'Humidity(%)',
    'pres': 'Pressure(hPa)'
})

weather_df['Date/Time'] = pd.to_datetime(weather_df['Date/Time'])

df = pd.merge(df, weather_df, on='Date/Time', how='left')
df = df.sort_values(ascending=True,by='Date/Time')
df.head()

In [ ]:
df['Temperature(°C)'].isna().sum()
df['Temperature(°C)'] = df['Temperature(°C)'].interpolate()

df['Humidity(%)'].isna().sum()
df['Humidity(%)'] = df['Humidity(%)'].interpolate()

df['Pressure(hPa)'].isna().sum()
df['Pressure(hPa)'] = df['Pressure(hPa)'].interpolate()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd  # Assuming you already have pandas imported

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(20, 18))

# Plot temperature on the first subplot
ax1.plot(df['Date/Time'], df['Temperature(°C)'], label='Temperature(°C)', color='#FF8C00')
ax1.set_xlabel('Date/Time', fontsize=16, color='red')
ax1.set_ylabel('Temperature (°C)', fontsize=16, color='red')
ax1.set_title('Temperature Over Time', fontsize=18, color='blue')
ax1.tick_params(axis='both', which='major', labelsize=12, rotation=45, color='orange')
ax1.legend()
ax1.grid(True)

# Plot humidity on the second subplot
ax2.plot(df['Date/Time'], df['Humidity(%)'], label='Humidity(%)', color='#9932CC')
ax2.set_xlabel('Date/Time', fontsize=16, color='red')
ax2.set_ylabel('Humidity (%)', fontsize=16, color='red')
ax2.set_title('Humidity Over Time', fontsize=18, color='blue')
ax2.tick_params(axis='both', which='major', labelsize=12, rotation=45, color='orange')
ax2.legend()
ax2.grid(True)

# Plot pressure on the third subplot
ax3.plot(df['Date/Time'], df['Pressure(hPa)'], label='Pressure(hPa)', color='#800000')
ax3.set_xlabel('Date/Time', fontsize=16, color='red')
ax3.set_ylabel('Pressure (hPa)', fontsize=16, color='red')
ax3.set_title('Pressure Over Time', fontsize=18, color='blue')
ax3.tick_params(axis='both', which='major', labelsize=12, rotation=45, color='orange')
ax3.legend()
ax3.grid(True)

# Plot wind speed on the fourth subplot
ax4.plot(df['Date/Time'], df['Wind Speed (m/s)'], label='Wind Speed (m/s)', color='Green')
ax4.set_xlabel('Date/Time', fontsize=16, color='red')
ax4.set_ylabel('Wind Speed (m/s)', fontsize=16, color='red')
ax4.set_title('Wind Speed Over Time', fontsize=18, color='blue')
ax4.tick_params(axis='both', which='major', labelsize=12, rotation=45, color='orange')
ax4.legend()
ax4.grid(True)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

## Understanding Yaw Systems

**A yaw system is a mechanism in a wind turbine that allows the turbine to rotate its nacelle (the housing for the generator and other components) to align the turbine blades with the prevailing wind direction. This ensures that the turbine captures the maximum amount of wind energy.**

**Determining Yaw System Existence**

- To determine if a yaw system exists in your data, you can analyze the relationship between wind direction and power output.

1. Plot a Scatter Plot:

- Create a scatter plot with wind direction on the x-axis and power output on the y-axis.
- If the data points cluster around a specific wind direction or a narrow range of directions, it suggests that the turbine might not have a yaw system or that it is malfunctioning.
- If the data points are scattered across a wide range of wind directions with relatively consistent power output, it indicates that the turbine likely has a functional yaw system.

2. Statistical Analysis:

- Calculate the correlation coefficient between wind direction and power output. A low correlation coefficient suggests that the yaw system is working effectively, as power output is relatively independent of wind direction.

**Assessing Wind Direction Coverage**

To determine if your data covers all wind directions, you can:

- Calculate the range of wind directions: Find the minimum and maximum wind directions in your dataset.
- Check for gaps: If the range is significantly smaller than 360 degrees, there might be gaps in your data coverage.
- Visualize wind direction distribution: Create a histogram or bar plot to see how evenly the data is distributed across different wind directions.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'Wind Direction' and 'Power Output' are your column names
df['Wind Direction (°)'] = df['Wind Direction (°)'].fillna(0)  # Handle missing values for wind direction
df['Wind Speed (m/s)'] = df['Wind Speed (m/s)'].fillna(0)  # Handle missing values for wind speed (if necessary)

# Plot scatter plot for wind direction vs. power output
plt.figure(figsize=(20, 6))  # Set figure size
plt.scatter(df['Wind Direction (°)'], df['LV ActivePower (kW)'], color='#FF8C00')
plt.xlabel('Wind Direction (degrees)', fontsize=16)  # Increase label font size
plt.ylabel('Power Output (kW)', fontsize=16)
plt.title('Wind Direction vs. Power Output', fontsize=20)  # Increase title font size
plt.grid(True)  # Add grid lines for better readability
plt.show()

# Calculate correlation coefficient
correlation = df['Wind Direction (°)'].corr(df['LV ActivePower (kW)'])
print('Correlation coefficient:', correlation)

# Calculate range of wind directions
wind_direction_range = df['Wind Direction (°)'].max() - df['Wind Direction (°)'].min()
print('Wind direction range:', wind_direction_range)

# Plot scatter plot for wind speed vs. power output
plt.figure(figsize=(20, 6))  # Set figure size
plt.scatter(df['Wind Speed (m/s)'], df['LV ActivePower (kW)'], color='#7FFF00')
plt.xlabel('Wind Speed (m/s)', fontsize=16)
plt.ylabel('LV ActivePower (kW)', fontsize=16)
plt.title('Wind Speed vs. Power Output', fontsize=20)
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'Wind Direction' and 'Hour' are your column names
df['Wind Direction (°)'] = df['Wind Direction (°)'].fillna(0)  # Handle missing values

# Bright color for the scatter plot
bright_color = '#FF8C00'  # Orange

# Plot wind direction vs. hour with transparency
plt.figure(figsize=(20, 6))  # Set figure size
plt.scatter(df['Wind Direction (°)'], df['Hour'], alpha=0.2, color=bright_color)
plt.title('Does the dataframe cover all wind directions?', fontsize=16)  # Increase title font size
plt.xlabel('Wind Direction (degrees)', fontsize=14)  # Increase label font size
plt.ylabel('Hour', fontsize=14)
plt.xticks(list(range(0, 365, 15)))  # Set x-axis ticks for every 15 degrees
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ensure cut_in_speed is defined
cut_in_speed = df[df['Theoretical_Power_Curve (KWh)'] > 0]['Wind Speed (m/s)'].min()

# Filter data based on cut-in speed
filtered_df = df[df['Wind Speed (m/s)'] > cut_in_speed]

# Create the plot
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: Wind Direction vs. Power Output
scatter1 = axes[0].scatter(df['Wind Direction (°)'], df['LV ActivePower (kW)'],
                           alpha=0.5, c=df['Wind Speed (m/s)'], cmap='viridis', s=2)
axes[0].set_title('Wind Direction vs. Power Output', fontsize=18, color='#DC143C')
axes[0].set_xlabel('Wind Direction (°)', fontsize=14, color='red')
axes[0].set_ylabel('Power Output (kW)', fontsize=14, color='red')
axes[0].set_xticks(list(range(0, 361, 45)))
axes[0].tick_params(axis='both', which='major', labelsize=12, colors='red')
axes[0].grid(True, alpha=0.3)
fig.colorbar(scatter1, ax=axes[0], label='Wind Speed (m/s)')

# Plot 2: Wind Direction vs. Wind Speed (filtered)
scatter2 = axes[1].scatter(filtered_df['Wind Direction (°)'], filtered_df['Wind Speed (m/s)'],
                           alpha=0.5, c=filtered_df['LV ActivePower (kW)'], cmap='plasma', s=2)
axes[1].set_title('Wind Direction vs. Wind Speed (Filtered)', fontsize=18, color='#FF8C00')
axes[1].set_xlabel('Wind Direction (°)', fontsize=14, color='red')
axes[1].set_ylabel('Wind Speed (m/s)', fontsize=14, color='red')
axes[1].set_xticks(list(range(0, 361, 45)))
axes[1].tick_params(axis='both', which='major', labelsize=12, colors='red')
axes[1].grid(True, alpha=0.3)
fig.colorbar(scatter2, ax=axes[1], label='Power Output (kW)')

# Adjust layout and display
plt.tight_layout()
plt.show()

# Additional analysis
def analyze_wind_direction(df):
    # Calculate average power output for each wind direction
    avg_power = df.groupby('Wind Direction (°)')['LV ActivePower (kW)'].mean()

    # Find the wind direction with maximum average power output
    max_power_direction = avg_power.idxmax()

    # Calculate the percentage of time wind comes from each direction
    wind_direction_counts = df['Wind Direction (°)'].value_counts(normalize=True) * 100

    # Find the most common wind direction
    most_common_direction = wind_direction_counts.idxmax()

    print(f"Wind direction with highest average power output: {max_power_direction:.2f}°")
    print(f"Most common wind direction: {most_common_direction:.2f}°")
    print("\nTop 5 most common wind directions:")
    print(wind_direction_counts.nlargest(5))

# Run the analysis
analyze_wind_direction(df)

The two scatter plots provide valuable insights into the relationship between wind direction, wind speed, and power output.

**Plot 1:** Wind Direction vs. Power Output

- **Data Distribution:** The data points are clustered around certain wind directions, indicating that the turbine is likely designed or optimized to perform better at those angles.
- **Average Power Output:** The "Wind direction with highest average power output" annotation suggests that the turbine achieves its highest average power at around 156.28 degrees.
- **Yaw System Assessment:** The relatively consistent power output across different wind directions, especially within the high-power clusters, suggests that the yaw system is functioning effectively.

**Plot 2:** Wind Direction vs. Wind Speed (Filtered)

- **Wind Speed Distribution:** The plot shows that higher wind speeds are concentrated within specific wind direction ranges (0°, 70°) and (190°, 210°). This indicates that the turbine's location or surrounding topography might influence wind patterns.
- **Yaw System Effectiveness:** The concentration of high wind speeds within specific wind direction ranges further supports the conclusion that the yaw system is working as intended, aligning the turbine with the prevailing wind direction.

**Key Observations and Insights:**

- **Optimal Wind Direction:** The turbine seems to be most efficient at wind directions around 156.28 degrees.
- **Yaw System Effectiveness:** The data suggests that the yaw system is functioning correctly, aligning the turbine with the direction of prevailing winds.
- **Wind Speed Distribution:** The concentration of higher wind speeds in specific wind direction ranges highlights the influence of geographical factors on wind patterns.

**Calculating Effective Theoretical Power:**

To calculate the effective theoretical power, you can follow these steps:

Determine Optimal Wind Direction: Based on your analysis, the optimal wind direction appears to be around 156.28 degrees.
Calculate Deviation: For each data point, calculate the deviation from the optimal wind direction:

In [ ]:
def calculate_deviation(wind_direction, optimal_angles=[60, 210]):
    """Calculates the minimum angular deviation between a wind direction and a list of optimal angles.

    Args:
        wind_direction (float): The wind direction in degrees.
        optimal_angles (list): A list of optimal wind directions in degrees.

    Returns:
        float: The minimum angular deviation in degrees.
    """

    deviations = [min(abs(wind_direction - angle), abs(wind_direction - angle + 360), abs(wind_direction - angle - 360)) for angle in optimal_angles]
    return min(deviations)

# Calculate the effective theoretical power
df['Effective Theoretical Power(kWh)'] = (100 - ((df['Wind Direction (°)'].apply(calculate_deviation) / 360) * 100)) * df['Theoretical_Power_Curve (KWh)']

df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.get_dummies(df, ['Season'])
df = df.drop(columns= 'Date/Time')
scaler = StandardScaler()
columns = df.columns

# Fit and transform the data
df = scaler.fit_transform(df)

# Convert the result back to a DataFrame
df = pd.DataFrame(df, columns = columns)

df = pd.get_dummies(df)
y = df['LV ActivePower (kW)']
X = df.drop(columns=['LV ActivePower (kW)'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Set figure size using rcParams (more persistent)
from pylab import rcParams
rcParams['figure.figsize'] = 20, 8

import matplotlib.pyplot as plt
import xgboost as xgb

# Train XGBoost model
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
# Train XGBoost model
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
importances = model.feature_importances_
feature_names = X.columns

# Plot feature importances
xgb.plot_importance(model)

# Increase text size for all elements
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel('Features', fontsize=16)
plt.ylabel('Feature Importance', fontsize=16)
plt.title('XGBoost Feature Importance', fontsize=18)

# Rotate x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Adjust layout for better precision
plt.tight_layout()
plt.show()
plt.show()

In [ ]:
%%time

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Assuming X and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models with hyperparameter ranges for tuning
models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'XGBoost': {
        'model': XGBRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5],
            'min_child_weight': [1, 5, 10]
        }
    },
    'LightGBM': {
        'model': LGBMRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [-1, 5, 10],
            'num_leaves': [31, 50, 100]
        }
    },
    'CatBoost': {
        'model': CatBoostRegressor(random_state=42, verbose=False),
        'params': {
            'iterations': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'depth': [4, 6, 8],
            'l2_leaf_reg': [1, 3, 5, 7, 9]
        }
    }
}

# Function to train and evaluate a model
def train_and_evaluate(model, params, X_train, y_train, X_test, y_test):
    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, params, n_iter=10, cv=3, n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)

    # Get the best model
    best_model = random_search.best_estimator_

    # Make predictions
    y_pred = best_model.predict(X_test)

    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return best_model, r2, rmse

# Train and evaluate models
results = []
best_models = {}

for name, model_info in models.items():
    print(f"Training {name}...")
    best_model, r2, rmse = train_and_evaluate(
        model_info['model'],
        model_info['params'],
        X_train_scaled,
        y_train,
        X_test_scaled,
        y_test
    )
    results.append({'Model-Name': name, 'R2_score': r2 * 100, 'RMSE': rmse})
    best_models[name] = best_model

# Create ensemble model
ensemble = VotingRegressor(
    estimators=[(name, model) for name, model in best_models.items()],
    weights=[1] * len(best_models)  # Equal weights for all models
)

# Train and evaluate ensemble model
print("Training Ensemble Model...")
ensemble.fit(X_train_scaled, y_train)
y_pred_ensemble = ensemble.predict(X_test_scaled)
r2_ensemble = r2_score(y_test, y_pred_ensemble)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))

results.append({'Model-Name': 'Ensemble', 'R2_score': r2_ensemble * 100, 'RMSE': rmse_ensemble})

# Create and sort results dataframe
models_df = pd.DataFrame(results)
models_df = models_df.sort_values("R2_score", ascending=False)

print("\nModel Performance:")
print(models_df)

# Print best model and its hyperparameters
best_model_name = models_df.iloc[0]['Model-Name']
if best_model_name != 'Ensemble':
    print(f"\nBest Model: {best_model_name}")
    print("Best Hyperparameters:")
    print(best_models[best_model_name].get_params())
else:
    print("\nBest Model: Ensemble")

print(f"\nBest R2 Score: {models_df.iloc[0]['R2_score']:.2f}%")
print(f"Best RMSE: {models_df.iloc[0]['RMSE']:.2f}")

In [ ]:
print("\nModel Performance:")
models_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for R2 Score
data_r2 = {
    "Model-Name": ["CatBoost", "LightGBM", "Ensemble", "GradientBoosting", "XGBoost", "Random Forest"],
    "R2_score": [98.54, 98.39, 98.27, 98.05, 97.61, 97.37]
}

# Data for RMSE
data_rmse = {
    "Model-Name": ["CatBoost", "LightGBM", "Ensemble", "GradientBoosting", "XGBoost", "Random Forest"],
    "RMSE": [0.12, 0.13, 0.13, 0.14, 0.15, 0.16]
}

# Create a single figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Function to create and customize bar plot
def create_bar_plot(ax, data, title, ylabel, colors):
    bars = ax.bar(data["Model-Name"], data[ylabel], color=colors)

    # Add labels on top of each bar
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}{"%" if ylabel == "R2_score" else ""}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')

    # Customize plot
    ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel("Model Name", fontsize=14, fontweight='bold', labelpad=10)
    ax.set_ylabel(ylabel, fontsize=14, fontweight='bold', labelpad=10)

    # Rotate x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

    # Adjust y-axis
    if ylabel == "R2_score":
        ax.set_ylim(97, 99)  # Adjusted to zoom in on the relevant range
    else:
        ax.set_ylim(0, max(data[ylabel]) * 1.2)  # Give some headroom above the highest bar

    # Add horizontal lines for better readability
    ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=0.7)

    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Increase tick label font size
    ax.tick_params(axis='both', which='major', labelsize=12)

# Plot R2 Score
colors_r2 = plt.cm.Pastel1(np.linspace(0, 1, 6))  # Using a colormap for consistency
create_bar_plot(ax1, data_r2, "Model Performance (R2 Score)", "R2_score", colors_r2)

# Plot RMSE
colors_rmse = plt.cm.Pastel2(np.linspace(0, 1, 6))  # Using a different colormap for RMSE
create_bar_plot(ax2, data_rmse, "Model Performance (RMSE)", "RMSE", colors_rmse)

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
print(f"\nBest R2 Score: {models_df.iloc[0]['R2_score']:.2f}%")

In [ ]:
print(f"Best RMSE: {models_df.iloc[0]['RMSE']:.2f}")